# Module 7: Linear Algebra with NumPy

NumPy contains the [`numpy.linalg`](https://docs.scipy.org/doc/numpy/reference/routines.linalg.html) package to solve matrix problems.

In [2]:
import numpy as np

In [3]:
help(np.linalg)

Help on package numpy.linalg in numpy:

NAME
    numpy.linalg

DESCRIPTION
    Core Linear Algebra Tools
    -------------------------
    Linear algebra basics:
    
    - norm            Vector or matrix norm
    - inv             Inverse of a square matrix
    - solve           Solve a linear system of equations
    - det             Determinant of a square matrix
    - lstsq           Solve linear least-squares problem
    - pinv            Pseudo-inverse (Moore-Penrose) calculated using a singular
                      value decomposition
    - matrix_power    Integer power of a square matrix
    
    Eigenvalues and decompositions:
    
    - eig             Eigenvalues and vectors of a square matrix
    - eigh            Eigenvalues and eigenvectors of a Hermitian matrix
    - eigvals         Eigenvalues of a square matrix
    - eigvalsh        Eigenvalues of a Hermitian matrix
    - qr              QR decomposition of a matrix
    - svd             Singular value decomposition 

### System of coupled linear equations
Solve the coupled system of linear equations of the general form

$$
\mathsf{A} \mathbf{x} = \mathbf{b}.
$$

where $\mathsf{A}$ is a *matrix* and $\mathbf{x}$, $\mathbf{b}$ are *vectors.*

In [4]:
A = np.array([
        [1, 0, 0],
        [0, 1, 0],
        [-1, 0, 2]
    ])
b = np.array([1, 0, 3])

What does this system of equations look like?

In [5]:
for i in range(A.shape[0]):
    terms = []
    for j in range(A.shape[1]):
        terms.append("{1:2} x[{0}]".format(j, A[i, j]))
    print(" + ".join(terms), "=", b[i])

 1 x[0] +  0 x[1] +  0 x[2] = 1
 0 x[0] +  1 x[1] +  0 x[2] = 0
-1 x[0] +  0 x[1] +  2 x[2] = 3


Now solve it with [`numpy.linalg.solve`](https://docs.scipy.org/doc/numpy/reference/generated/numpy.linalg.solve.html#numpy.linalg.solve):

In [6]:
x = np.linalg.solve(A, b)
print(x)

[1. 0. 2.]


Test that it satisfies the original equation:
$$
\mathsf{A} \mathbf{x} - \mathbf{b} = 0
$$

In [7]:
np.dot(A, x) - b

array([0., 0., 0.])

### Matrix inverse
In order to solve directly we need the inverse of $\mathsf{A}$:
$$
\mathsf{A}\mathsf{A}^{-1} = \mathsf{A}^{-1}\mathsf{A} = \mathsf{1}
$$

Then
$$
\mathbf{x} = \mathsf{A}^{-1} \mathbf{b}
$$

If the inverse exists, [`numpy.linalg.inv()`](https://docs.scipy.org/doc/numpy/reference/generated/numpy.linalg.inv.html#numpy.linalg.inv) can calculate it:

In [8]:
Ainv = np.linalg.inv(A)
print(Ainv)

[[1.  0.  0. ]
 [0.  1.  0. ]
 [0.5 0.  0.5]]


Check that it behaves like an inverse:
$$
\mathsf{A}^{-1} \mathsf{A} = \mathsf{A} \mathsf{A}^{-1} = \mathsf{1}
$$

In [9]:
Ainv.dot(A)

array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.]])

In [10]:
A.dot(Ainv)

array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.]])

Now solve the coupled equations directly:

In [11]:
Ainv.dot(b)

array([1., 0., 2.])

### Eigenvalue problems
The equation

\begin{gather}
\mathsf{A} \mathbf{x}_i = \lambda_i \mathbf{x}_i
\end{gather}

is the **eigenvalue problem** and a solution provides the eigenvalues $\lambda_i$ and corresponding eigenvectors $x_i$ that satisfy the equation.

#### Example 1: Principal axes of a square
The principle axes of the [moment of inertia tensor](https://en.wikipedia.org/wiki/Moment_of_inertia#The_inertia_tensor) are defined through the eigenvalue problem

$$
\mathsf{I} \boldsymbol{\omega}_i = \lambda_i \boldsymbol{\omega}_i
$$

The principal axes are the vectors $\boldsymbol{\omega}_i$.

For a flat solid square, the tensor of inertia is
$$
\mathsf{I} = \begin{pmatrix} \frac{2}{3} & -\frac{1}{4} \\ -\frac{1}{4} & \frac{2}{3}\end{pmatrix}
$$

In [12]:
Isquare = np.array([[2/3, -1/4], [-1/4, 2/3]])

Solve the eigenvalue problem

In [13]:
lambdas, omegas = np.linalg.eig(Isquare)

In [14]:
lambdas

array([0.91666667, 0.41666667])

In [15]:
omegas

array([[ 0.70710678,  0.70710678],
       [-0.70710678,  0.70710678]])

Note that the eigenvectors are `omegas[:, i]`! You can transpose so that axis 0 is the eigenvector index:

In [16]:
omegas.T

array([[ 0.70710678, -0.70710678],
       [ 0.70710678,  0.70710678]])

Test:
$$
(\mathsf{I} - \lambda_i \mathsf{1}) \boldsymbol{\omega}_i = 0
$$
(The identity matrix can be generated with `np.identity(2)`.)

In [ ]:
(Isquare - lambdas[0]*np.identity(2)).dot(omegas[:, 0])

Check $\lambda_1$

In [17]:
idx = 0
(Isquare - lambdas[idx]*np.identity(2)).dot(omegas.T[idx])

array([0., 0.])

Check $\lambda_2$

In [18]:
idx = 1
(Isquare - lambdas[idx]*np.identity(2)).dot(omegas.T[idx])

array([0., 0.])

#### Example 2: Spin in a magnetic field
In quantum mechanics, a spin 1/2 particle is represented by a spinor $\chi$, a 2-component vector. The Hamiltonian operator for a stationary spin 1/2 particle in a homogenous magnetic field $B_y$ is 

$$
\mathsf{H} = -\gamma \mathsf{S}_y B_y = -\gamma B_y \frac{\hbar}{2} \mathsf{\sigma_y} 
   = \hbar \omega \left( \begin{array}{cc} 0 & -i \\ i & 0 \end{array}\right)
$$

Determine the *eigenvalues* and *eigenstates*
$$
\mathsf{H} \mathbf{\chi} = E \mathbf{\chi}
$$
of the spin 1/2 particle.

(To make this a purely numerical problem, divide through by $\hbar\omega$, i.e. calculate $E/\hbar\omega$.)

Determine the *eigenvalues* $E_i$ and *eigenstates* $\chi_i$
$$
\mathsf{H} \mathbf{\chi}_i = E_i \mathbf{\chi}_i
$$

In [20]:
sigma_y = np.array([[0, -1j], [1j, 0]])
E, chis = np.linalg.eig(sigma_y)
print(E)
print(chis.T)

[ 1.+0.j -1.+0.j]
[[-0.        -0.70710678j  0.70710678+0.j        ]
 [ 0.70710678+0.j          0.        -0.70710678j]]


Normalize the eigenvectors:
$$
\hat\chi = \frac{1}{\sqrt{\chi^\dagger \cdot \chi}} \chi
$$

In [21]:
chi1 = chis.T[0]
print(chi1)
norm = np.dot(chi1.conjugate(), chi1)
chi1_hat = chi1/np.sqrt(norm)
print(chi1_hat)

[-0.        -0.70710678j  0.70710678+0.j        ]
[-0.        -0.70710678j  0.70710678+0.j        ]


In [ ]:
norm

... they were already normalized.